## Installing packages

In [1]:
# Linking to drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# Loading the drive
import os
os.chdir("/content/gdrive/Shareddrives/Low_Budget_MT")

In [3]:
#! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install . --use-feature=in-tree-build

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt
     |████████████████████████████████| 116 kB 34.9 MB/s 
     |████████████████████████████████| 507 kB 59.1 MB/s 
     |████████████████████████████████| 264 kB 63.5 MB/s 
     |████████████████████████████████| 103 kB 46.4 MB/s 
     |████████████████████████████████| 57 kB 6.6 MB/s 
     |████████████████████████████████| 843 kB 58.3 MB/s 
  Created wheel for joeynmt: filename=joeynmt-1.5.1-py3-none-any.whl size=86509 sha256=fd2504d144fb66b040d8170a3c7610c1a85f9773c3748c33cd3b7cf5c5403664
  Stored in directory: /tmp/pip-ephem-wheel-cache-ryeofk4i/wheels/ec/b0/4e/e15e388dd23d01f1c2b4065e1bb36099e5e22abe4f7147f706
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68439 sha256=557dd98bb2b7dce4fdc8e80b42882c61a2fc98c5c450d0c6945ef8b435145401
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d

In [4]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 28.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [5]:
# Importing needed libraries for preprocessing and visualization
from my_helpers import rttl_data_load, split_srctgt, generating_BPE, run_model, rttl_scoring, nrttl_data_load

import warnings
warnings.filterwarnings('ignore')

# Kinyarwanda K1

## Iteration 5

### Preprocessing

In [ ]:
rttl_data_load('rwtrain.csv', 'rw', 'rttl', 'k1', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw/k1
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/rttl
(30000, 2)
{1: 11727, 2: 4801, 3: 2396, 4: 821, 5: 198, 6: 41, 7: 12, 9: 3, 8: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
                                     source_sentence  \
0  Likewise , hypocrisy in religion only proves t...   

                                     target_sentence  \
0  Urwo rugero rugaragaza ko Imana itemera uburya...   

                                         translation    scores  tgt_len dist  
0  Urwo rugero rugaragaza ko Imana itemera uburya... -2.497851       18    2  
(20000, 2) (80000, 2)


In [ ]:
import pandas as pd
def split_srctgt(train, source_language, target_language, query, k, iteration):

  path = os.getcwd() 
  data_path = 'data/'+target_language+'/'+k+'/'+query
  
  os.chdir(os.path.join(path, data_path))
  if not os.path.exists(iteration):
    os.mkdir(iteration)
  os.chdir(iteration)
  # Splitting train and validation set
  train = pd.read_csv(path+train)
  print(train.shape)
  #num_valid = 1000

  # dev = train.tail(num_valid) 
  # stripped = train.drop(train.tail(num_valid).index)

  # Creating files: Train
  with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
    for index, row in train.iterrows():
      src_file.write(str(row["source_sentence"])+"\n")
      trg_file.write(str(row["target_sentence"])+"\n")

  # Dev   
  # with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  #   for index, row in dev.iterrows():
  #     src_file.write(row["source_sentence"]+"\n")
  #     trg_file.write(row["target_sentence"]+"\n")

  os.chdir(path)

In [ ]:
split_srctgt('/data/rw/k1/rttl/rttl_train.csv','en', 'rw', 'rttl', 'k1', '2')

(50000, 2)


In [ ]:
generating_BPE('en', 'rw', 'rttl', 'k1', '2')

### Training $M$

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k1', '2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enrw2_rttl.yaml

2022-09-03 21:57:32,209 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-03 21:57:32,466 - INFO - joeynmt.data - Loading training data...
2022-09-03 21:57:33,221 - INFO - joeynmt.data - Building vocabulary...
2022-09-03 21:57:33,518 - INFO - joeynmt.data - Loading dev data...
2022-09-03 21:57:33,535 - INFO - joeynmt.data - Loading test data...
2022-09-03 21:57:33,903 - INFO - joeynmt.data - Data loaded.
2022-09-03 21:57:33,903 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-03 21:57:34,117 - INFO - joeynmt.model - Enc-dec model built.
2022-09-03 21:57:37,836 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-03 21:57:38,174 - INFO - joeynmt.training - Total params: 12152832
2022-09-03 21:57:41,837 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_rttl
2022-09-03 21:57:41,837 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-09-03 21:57:41,837 - INFO - joeynmt.helpers -    

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k1/transformer_reverse_enes2_rttl/validations.txt"

Steps: 600	Loss: 191759.65625	PPL: 118.52570	bleu: 0.08312	LR: 0.00100000	*
Steps: 1200	Loss: 168775.37500	PPL: 66.87222	bleu: 0.78378	LR: 0.00100000	*
Steps: 1800	Loss: 154159.00000	PPL: 46.47023	bleu: 1.69323	LR: 0.00100000	*
Steps: 2400	Loss: 145812.25000	PPL: 37.74919	bleu: 2.38650	LR: 0.00100000	*
Steps: 3000	Loss: 138743.32812	PPL: 31.65627	bleu: 3.14686	LR: 0.00100000	*
Steps: 3600	Loss: 133185.12500	PPL: 27.56447	bleu: 3.77283	LR: 0.00100000	*
Steps: 4200	Loss: 128718.53125	PPL: 24.66296	bleu: 3.78458	LR: 0.00100000	*
Steps: 4800	Loss: 125177.82812	PPL: 22.58155	bleu: 4.60964	LR: 0.00100000	*
Steps: 5400	Loss: 121078.46094	PPL: 20.39015	bleu: 5.61840	LR: 0.00100000	*
Steps: 6000	Loss: 118367.15625	PPL: 19.05894	bleu: 5.97726	LR: 0.00100000	*
Steps: 6600	Loss: 115526.86719	PPL: 17.75751	bleu: 6.64065	LR: 0.00100000	*
Steps: 7200	Loss: 112821.75000	PPL: 16.60073	bleu: 7.08202	LR: 0.00100000	*
Steps: 7800	Loss: 109916.11719	PPL: 15.44201	bleu: 8.27100	LR: 0.00100000	*
Steps: 8400	

In [ ]:
# Reloading configuration file
ckpt_number = 28800
config = run_model('es', 'es','en','transformer_reverse_enes2_rttl','rttl', 'k1', '2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enes2_rttl/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k1/transformer_reverse_enes2_rttl"', f'model_dir: "models/k1/transformer_reverse_enes2_rttl_reload"').replace(
        f'epochs: 60', f'epochs: 34')
        
with open("joeynmt/configs/k1/transformer_reverse_enes2_rttl_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enes2_rttl_reload.yaml

2022-09-02 04:29:46,885 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-02 04:29:47,995 - INFO - joeynmt.data - Loading training data...
2022-09-02 04:29:54,563 - INFO - joeynmt.data - Building vocabulary...
2022-09-02 04:29:55,783 - INFO - joeynmt.data - Loading dev data...
2022-09-02 04:29:57,922 - INFO - joeynmt.data - Loading test data...
2022-09-02 04:30:02,290 - INFO - joeynmt.data - Data loaded.
2022-09-02 04:30:02,291 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-02 04:30:02,492 - INFO - joeynmt.model - Enc-dec model built.
2022-09-02 04:30:06,780 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-02 04:30:07,089 - INFO - joeynmt.training - Total params: 12162048
2022-09-02 04:30:10,486 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enes2_rttl/28800.ckpt
2022-09-02 04:30:14,535 - INFO - joeynmt.helpers -                           cfg.name : tr

## Iteration 2

### Preprocessing

In [ ]:
rttl_data_load('rwtrain.csv', 'rw', 'rttl', 'k1', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw/k1
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/rttl
(30000, 2)
{1: 2932, 2: 1200, 3: 599, 4: 205, 5: 50, 6: 10, 7: 3, 9: 1, 8: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
                                     source_sentence  \
0  Likewise , hypocrisy in religion only proves t...   

                                     target_sentence  \
0  Urwo rugero rugaragaza ko Imana itemera uburya...   

                                         translation    scores  tgt_len dist  
0  Urwo rugero rugaragaza ko Imana itemera uburya... -2.497851       18    2  
(5000, 2) (95000, 2)


In [ ]:
split_srctgt('/data/rw/k1/rttl/rttl_train.csv','en', 'rw', 'rttl', 'k1', '2')

(35000, 2)


In [ ]:
generating_BPE('en', 'rw', 'rttl', 'k1', '2')

### Training $M$

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k1', '2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enrw2_rttl.yaml

2022-10-08 20:00:26,765 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-08 20:00:27,716 - INFO - joeynmt.data - Loading training data...
2022-10-08 20:00:28,152 - INFO - joeynmt.data - Building vocabulary...
2022-10-08 20:00:28,392 - INFO - joeynmt.data - Loading dev data...
2022-10-08 20:00:28,407 - INFO - joeynmt.data - Loading test data...
2022-10-08 20:00:28,713 - INFO - joeynmt.data - Data loaded.
2022-10-08 20:00:28,714 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-08 20:00:28,911 - INFO - joeynmt.model - Enc-dec model built.
2022-10-08 20:00:32,062 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-08 20:00:32,296 - INFO - joeynmt.training - Total params: 12146432
2022-10-08 20:00:35,811 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_rttl
2022-10-08 20:00:35,811 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-08 20:00:35,812 - INFO - joeynmt.helpers -    

### Training $M_{rev}$

In [ ]:
# Swahili to English
run_model('rw', 'en','rw','transformer_enrw2_rttl','rttl', 'k1', '2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_enrw2_rttl.yaml

2022-10-08 22:51:11,001 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-08 22:51:11,037 - INFO - joeynmt.data - Loading training data...
2022-10-08 22:51:11,476 - INFO - joeynmt.data - Building vocabulary...
2022-10-08 22:51:11,713 - INFO - joeynmt.data - Loading dev data...
2022-10-08 22:51:11,728 - INFO - joeynmt.data - Loading test data...
2022-10-08 22:51:12,023 - INFO - joeynmt.data - Data loaded.
2022-10-08 22:51:12,024 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-08 22:51:12,223 - INFO - joeynmt.model - Enc-dec model built.
2022-10-08 22:51:14,310 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-08 22:51:14,779 - INFO - joeynmt.training - Total params: 12146432
2022-10-08 22:51:18,168 - INFO - joeynmt.helpers -                           cfg.name : transformer_enrw2_rttl
2022-10-08 22:51:18,168 - INFO - joeynmt.helpers -                       cfg.data.src : en
2022-10-08 22:51:18,168 - INFO - joeynmt.helpers -            

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k1/transformer_enrw2_rttl/validations.txt"

Steps: 600	Loss: 133234.17188	PPL: 115.14118	bleu: 0.06521	LR: 0.00100000	*
Steps: 1200	Loss: 117743.03125	PPL: 66.30882	bleu: 0.87088	LR: 0.00100000	*
Steps: 1800	Loss: 107375.82812	PPL: 45.83350	bleu: 1.32312	LR: 0.00100000	*
Steps: 2400	Loss: 101103.64844	PPL: 36.65623	bleu: 1.55040	LR: 0.00100000	*
Steps: 3000	Loss: 96671.56250	PPL: 31.30258	bleu: 2.69028	LR: 0.00100000	*
Steps: 3600	Loss: 92761.27344	PPL: 27.23235	bleu: 2.93014	LR: 0.00100000	*
Steps: 4200	Loss: 89981.05469	PPL: 24.66455	bleu: 3.34814	LR: 0.00100000	*
Steps: 4800	Loss: 87385.14062	PPL: 22.48601	bleu: 3.98976	LR: 0.00100000	*
Steps: 5400	Loss: 85304.69531	PPL: 20.87980	bleu: 4.57874	LR: 0.00100000	*
Steps: 6000	Loss: 83766.28125	PPL: 19.76633	bleu: 4.43067	LR: 0.00100000	*
Steps: 6600	Loss: 82420.69531	PPL: 18.84121	bleu: 4.93170	LR: 0.00100000	*
Steps: 7200	Loss: 80928.19531	PPL: 17.86564	bleu: 5.55432	LR: 0.00100000	*
Steps: 7800	Loss: 79862.06250	PPL: 17.19986	bleu: 5.94090	LR: 0.00100000	*
Steps: 8400	Loss: 792

In [ ]:
# Reloading configuration file
ckpt_number = 12600
config = run_model('rw', 'en','rw','transformer_enrw2_rttl','rttl', 'k1', '2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_enrw2_rttl/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k1/transformer_enrw2_rttl"', f'model_dir: "models/k1/transformer_enrw2_rttl_reload"').replace(
        f'epochs: 60', f'epochs: 42')
        
with open("joeynmt/configs/k1/transformer_enrw2_rttl_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k1/transformer_enrw2_rttl_reload.yaml

2022-10-09 18:33:38,564 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-09 18:33:39,550 - INFO - joeynmt.data - Loading training data...
2022-10-09 18:33:45,491 - INFO - joeynmt.data - Building vocabulary...
2022-10-09 18:33:46,381 - INFO - joeynmt.data - Loading dev data...
2022-10-09 18:33:47,667 - INFO - joeynmt.data - Loading test data...
2022-10-09 18:33:49,601 - INFO - joeynmt.data - Data loaded.
2022-10-09 18:33:49,601 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-09 18:33:49,803 - INFO - joeynmt.model - Enc-dec model built.
2022-10-09 18:33:52,358 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-09 18:33:52,849 - INFO - joeynmt.training - Total params: 12146432
2022-10-09 18:33:56,410 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_enrw2_rttl/12600.ckpt
2022-10-09 18:34:01,110 - INFO - joeynmt.helpers -                           cfg.name : transforme

### Scoring

In [ ]:
rttl_scoring('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/rttl/rttl_rem.csv', 'en', 'rw','rttl', 'k1', '2','transformer_reverse_enrw2_rttl','transformer_enrw2_rttl_reload')

(95000, 2)
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/rttl/2
/content/gdrive/Shareddrives/Low_Budget_MT
Starting here
95000 95000 95000
95000
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k1/rttl/2
/content/gdrive/Shareddrives/Low_Budget_MT
doing scoring
95000 95000


# Kinyarwanda K2

## Iteration 1

### Training $M_{rev}$

In [ ]:
# English to Kinyarwanda
run_model('rw','en', 'rw','transformer_enrw1','random', 'k2', '1')
! cd joeynmt; python3 -m joeynmt train configs/k2/transformer_enrw1.yaml

2022-09-27 09:28:41,626 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-27 09:28:41,661 - INFO - joeynmt.data - Loading training data...
2022-09-27 09:28:42,054 - INFO - joeynmt.data - Building vocabulary...
2022-09-27 09:28:42,278 - INFO - joeynmt.data - Loading dev data...
2022-09-27 09:28:42,299 - INFO - joeynmt.data - Loading test data...
2022-09-27 09:28:42,595 - INFO - joeynmt.data - Data loaded.
2022-09-27 09:28:42,596 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-27 09:28:42,793 - INFO - joeynmt.model - Enc-dec model built.
2022-09-27 09:28:44,788 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-27 09:28:45,241 - INFO - joeynmt.training - Total params: 12141312
2022-09-27 09:28:48,637 - INFO - joeynmt.helpers -                           cfg.name : transformer_enrw1
2022-09-27 09:28:48,637 - INFO - joeynmt.helpers -                       cfg.data.src : en
2022-09-27 09:28:48,638 - INFO - joeynmt.helpers -                 

### Scoring

In [ ]:
!cp -a /source/. /dest/

In [ ]:
rttl_scoring('/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/rem.csv', 'en', 'rw','rttl','k2','1','transformer_reverse_enrw1','transformer_enrw1')

(100000, 2)


FileNotFoundError: ignored

## Iteration 5

### Preprocessing

In [ ]:
rttl_data_load('rwtrain.csv', 'rw', 'rttl', 'k2', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw/k2
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/rttl
(30000, 2)
{1: 11725, 2: 4746, 3: 2426, 4: 843, 5: 203, 6: 40, 7: 11, 8: 3, 9: 3, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
                                     source_sentence  \
0  So to delight in God’s law means having a deep...   

                                     target_sentence  \
0  Ubwo rero , kwishimira amategeko y’Imana bisob...   

                                         translation    scores  tgt_len dist  
0  Ubwo rero , kwishimira amategeko y’Imana bisob... -3.072932       20    2  
(20000, 2) (80000, 2)


In [ ]:
split_srctgt('/data/rw/k2/rttl/rttl_train.csv','en', 'rw', 'rttl', 'k2', '2')

(50000, 2)


In [ ]:
generating_BPE('en', 'rw', 'rttl', 'k2', '2')

### Training $M$

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k2', '2')
! cd joeynmt; python3 -m joeynmt train configs/k2/transformer_reverse_enrw2_rttl.yaml

2022-10-15 01:06:26,491 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 01:06:27,245 - INFO - joeynmt.data - Loading training data...
2022-10-15 01:06:27,927 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 01:06:28,167 - INFO - joeynmt.data - Loading dev data...
2022-10-15 01:06:28,182 - INFO - joeynmt.data - Loading test data...
2022-10-15 01:06:28,547 - INFO - joeynmt.data - Data loaded.
2022-10-15 01:06:28,547 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 01:06:28,745 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 01:06:32,230 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 01:06:32,517 - INFO - joeynmt.training - Total params: 12153856
2022-10-15 01:06:35,997 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_rttl
2022-10-15 01:06:35,997 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 01:06:35,997 - INFO - joeynmt.helpers -    

## Iteration 2

### Preprocessing

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/nrttl/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/rttl/1/

In [ ]:
rttl_data_load('rwtrain.csv', 'rw', 'rttl', 'k2', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw/k2
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k2/rttl
(30000, 2)
{1: 2931, 2: 1186, 3: 606, 4: 211, 5: 51, 6: 10, 7: 3, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
                                     source_sentence  \
0  So to delight in God’s law means having a deep...   

                                     target_sentence  \
0  Ubwo rero , kwishimira amategeko y’Imana bisob...   

                                         translation    scores  tgt_len dist  
0  Ubwo rero , kwishimira amategeko y’Imana bisob... -3.072932       20    2  
(5000, 2) (95000, 2)


In [ ]:
split_srctgt('/data/rw/k2/rttl/rttl_train.csv','en', 'rw', 'rttl', 'k2', '2')

(35000, 2)


In [ ]:
generating_BPE('en', 'rw', 'rttl', 'k2', '2')

### Training $M$

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k2', '2')
! cd joeynmt; python3 -m joeynmt train configs/k2/transformer_reverse_enrw2_rttl.yaml

2022-10-09 22:39:33,508 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-09 22:39:35,262 - INFO - joeynmt.data - Loading training data...
2022-10-09 22:39:35,700 - INFO - joeynmt.data - Building vocabulary...
2022-10-09 22:39:35,937 - INFO - joeynmt.data - Loading dev data...
2022-10-09 22:39:35,953 - INFO - joeynmt.data - Loading test data...
2022-10-09 22:39:36,249 - INFO - joeynmt.data - Data loaded.
2022-10-09 22:39:36,249 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-09 22:39:36,455 - INFO - joeynmt.model - Enc-dec model built.
2022-10-09 22:39:40,054 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-09 22:39:40,506 - INFO - joeynmt.training - Total params: 12149504
2022-10-09 22:39:44,025 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_rttl
2022-10-09 22:39:44,025 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-09 22:39:44,025 - INFO - joeynmt.helpers -    

### Training $M_{rev}$

In [ ]:
# Swahili to English
run_model('rw', 'en','rw','transformer_enrw2_rttl','rttl', 'k1', '2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_enrw2_rttl.yaml

2022-10-08 22:51:11,001 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-08 22:51:11,037 - INFO - joeynmt.data - Loading training data...
2022-10-08 22:51:11,476 - INFO - joeynmt.data - Building vocabulary...
2022-10-08 22:51:11,713 - INFO - joeynmt.data - Loading dev data...
2022-10-08 22:51:11,728 - INFO - joeynmt.data - Loading test data...
2022-10-08 22:51:12,023 - INFO - joeynmt.data - Data loaded.
2022-10-08 22:51:12,024 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-08 22:51:12,223 - INFO - joeynmt.model - Enc-dec model built.
2022-10-08 22:51:14,310 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-08 22:51:14,779 - INFO - joeynmt.training - Total params: 12146432
2022-10-08 22:51:18,168 - INFO - joeynmt.helpers -                           cfg.name : transformer_enrw2_rttl
2022-10-08 22:51:18,168 - INFO - joeynmt.helpers -                       cfg.data.src : en
2022-10-08 22:51:18,168 - INFO - joeynmt.helpers -            

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k1/transformer_enrw2_rttl/validations.txt"

Steps: 600	Loss: 133234.17188	PPL: 115.14118	bleu: 0.06521	LR: 0.00100000	*
Steps: 1200	Loss: 117743.03125	PPL: 66.30882	bleu: 0.87088	LR: 0.00100000	*
Steps: 1800	Loss: 107375.82812	PPL: 45.83350	bleu: 1.32312	LR: 0.00100000	*
Steps: 2400	Loss: 101103.64844	PPL: 36.65623	bleu: 1.55040	LR: 0.00100000	*
Steps: 3000	Loss: 96671.56250	PPL: 31.30258	bleu: 2.69028	LR: 0.00100000	*
Steps: 3600	Loss: 92761.27344	PPL: 27.23235	bleu: 2.93014	LR: 0.00100000	*
Steps: 4200	Loss: 89981.05469	PPL: 24.66455	bleu: 3.34814	LR: 0.00100000	*
Steps: 4800	Loss: 87385.14062	PPL: 22.48601	bleu: 3.98976	LR: 0.00100000	*
Steps: 5400	Loss: 85304.69531	PPL: 20.87980	bleu: 4.57874	LR: 0.00100000	*
Steps: 6000	Loss: 83766.28125	PPL: 19.76633	bleu: 4.43067	LR: 0.00100000	*
Steps: 6600	Loss: 82420.69531	PPL: 18.84121	bleu: 4.93170	LR: 0.00100000	*
Steps: 7200	Loss: 80928.19531	PPL: 17.86564	bleu: 5.55432	LR: 0.00100000	*
Steps: 7800	Loss: 79862.06250	PPL: 17.19986	bleu: 5.94090	LR: 0.00100000	*
Steps: 8400	Loss: 792

In [ ]:
# Reloading configuration file
ckpt_number = 12600
config = run_model('rw', 'en','rw','transformer_enrw2_rttl','rttl', 'k1', '2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_enrw2_rttl/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k1/transformer_enrw2_rttl"', f'model_dir: "models/k1/transformer_enrw2_rttl_reload"').replace(
        f'epochs: 60', f'epochs: 42')
        
with open("joeynmt/configs/k1/transformer_enrw2_rttl_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k1/transformer_enrw2_rttl_reload.yaml

2022-10-09 18:33:38,564 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-09 18:33:39,550 - INFO - joeynmt.data - Loading training data...
2022-10-09 18:33:45,491 - INFO - joeynmt.data - Building vocabulary...
2022-10-09 18:33:46,381 - INFO - joeynmt.data - Loading dev data...
2022-10-09 18:33:47,667 - INFO - joeynmt.data - Loading test data...
2022-10-09 18:33:49,601 - INFO - joeynmt.data - Data loaded.
2022-10-09 18:33:49,601 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-09 18:33:49,803 - INFO - joeynmt.model - Enc-dec model built.
2022-10-09 18:33:52,358 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-09 18:33:52,849 - INFO - joeynmt.training - Total params: 12146432
2022-10-09 18:33:56,410 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_enrw2_rttl/12600.ckpt
2022-10-09 18:34:01,110 - INFO - joeynmt.helpers -                           cfg.name : transforme

# Kinyarwanda K3

## Iteration 1

### Training $M_{rev}$

In [ ]:
# English to Kinyarwanda
run_model('rw','en', 'rw','transformer_enrw1','random', 'k3', '1')
! cd joeynmt; python3 -m joeynmt train configs/k3/transformer_enrw1.yaml

2022-09-28 07:15:57,115 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-28 07:15:57,408 - INFO - joeynmt.data - Loading training data...
2022-09-28 07:16:00,110 - INFO - joeynmt.data - Building vocabulary...
2022-09-28 07:16:00,797 - INFO - joeynmt.data - Loading dev data...
2022-09-28 07:16:01,577 - INFO - joeynmt.data - Loading test data...
2022-09-28 07:16:02,941 - INFO - joeynmt.data - Data loaded.
2022-09-28 07:16:02,941 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-28 07:16:03,141 - INFO - joeynmt.model - Enc-dec model built.
2022-09-28 07:16:06,092 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-28 07:16:06,578 - INFO - joeynmt.training - Total params: 12141056
2022-09-28 07:16:09,950 - INFO - joeynmt.helpers -                           cfg.name : transformer_enrw1
2022-09-28 07:16:09,951 - INFO - joeynmt.helpers -                       cfg.data.src : en
2022-09-28 07:16:09,951 - INFO - joeynmt.helpers -                 

## Iteration 5

In [ ]:
# Importing needed libraries for preprocessing and visualization
from my_helpers import rttl_data_load, split_srctgt, generating_BPE, run_model, rttl_scoring, nrttl_data_load

import warnings
warnings.filterwarnings('ignore')

### Preprocessing

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/nrttl/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/rttl/1/

In [ ]:
rttl_data_load('rwtrain.csv', 'rw', 'rttl', 'k3', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw/k3
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/rttl
(30000, 2)
{1: 11750, 2: 4753, 3: 2395, 4: 839, 5: 197, 6: 47, 7: 13, 8: 3, 9: 3, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
                                     source_sentence  \
0  Saint Peter calls on all Christians to be prie...   

                                     target_sentence  \
0  Mutagatifu Petero ahamagarira Abakristo bose k...   

                                         translation    scores  tgt_len dist  
0  Mutagatifu Petero ahamagarira Abakristo bose k... -2.583092       22    3  
(20000, 2) (80000, 2)


In [ ]:
split_srctgt('/data/rw/k3/rttl/rttl_train.csv','en', 'rw', 'rttl', 'k3', '2')

(50000, 2)


In [ ]:
generating_BPE('en', 'rw', 'rttl', 'k3', '2')

### Training $M$

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k3', '2')
! cd joeynmt; python3 -m joeynmt train configs/k3/transformer_reverse_enrw2_rttl.yaml

2022-10-15 01:08:34,914 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 01:08:35,776 - INFO - joeynmt.data - Loading training data...
2022-10-15 01:08:36,463 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 01:08:36,719 - INFO - joeynmt.data - Loading dev data...
2022-10-15 01:08:36,749 - INFO - joeynmt.data - Loading test data...
2022-10-15 01:08:37,117 - INFO - joeynmt.data - Data loaded.
2022-10-15 01:08:37,117 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 01:08:37,328 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 01:08:41,709 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 01:08:41,954 - INFO - joeynmt.training - Total params: 12154112
2022-10-15 01:08:45,571 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_rttl
2022-10-15 01:08:45,571 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 01:08:45,571 - INFO - joeynmt.helpers -    

In [6]:
!cd joeynmt; python -m joeynmt test 'models/k3/transformer_reverse_enrw2_rttl/config.yaml'

2022-10-17 09:16:57,699 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-17 09:16:59,800 - INFO - joeynmt.data - Building vocabulary...
2022-10-17 09:17:03,667 - INFO - joeynmt.data - Loading dev data...
2022-10-17 09:17:05,459 - INFO - joeynmt.data - Loading test data...
2022-10-17 09:17:08,981 - INFO - joeynmt.data - Data loaded.
2022-10-17 09:17:09,039 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2022-10-17 09:17:09,039 - INFO - joeynmt.prediction - Loading model from models/k3/transformer_reverse_enrw2_rttl/57000.ckpt
2022-10-17 09:17:17,712 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-17 09:17:17,910 - INFO - joeynmt.model - Enc-dec model built.
2022-10-17 09:17:17,976 - INFO - joeynmt.prediction - Decoding on dev set (/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k3/rttl/2/dev.bpe.en)...
2022-10-17 09:18:05,836 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
20

# Kinyarwanda K4

## Iteration 1

### Training $M_{rev}$

In [ ]:
# English to Kinyarwanda
run_model('rw','en', 'rw','transformer_enrw1','random', 'k4', '1')
! cd joeynmt; python3 -m joeynmt train configs/k4/transformer_enrw1.yaml

2022-09-28 09:29:10,065 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-28 09:29:10,101 - INFO - joeynmt.data - Loading training data...
2022-09-28 09:29:12,672 - INFO - joeynmt.data - Building vocabulary...
2022-09-28 09:29:13,343 - INFO - joeynmt.data - Loading dev data...
2022-09-28 09:29:14,140 - INFO - joeynmt.data - Loading test data...
2022-09-28 09:29:15,369 - INFO - joeynmt.data - Data loaded.
2022-09-28 09:29:15,370 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-28 09:29:15,588 - INFO - joeynmt.model - Enc-dec model built.
2022-09-28 09:29:17,829 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-28 09:29:18,258 - INFO - joeynmt.training - Total params: 12141312
2022-09-28 09:29:21,585 - INFO - joeynmt.helpers -                           cfg.name : transformer_enrw1
2022-09-28 09:29:21,585 - INFO - joeynmt.helpers -                       cfg.data.src : en
2022-09-28 09:29:21,586 - INFO - joeynmt.helpers -                 

## Iteration 5

### Preprocessing

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/nrttl/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/rttl/1/

In [ ]:
rttl_data_load('rwtrain.csv', 'rw', 'rttl', 'k4', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw/k4
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k4/rttl
(30000, 2)
{1: 11732, 2: 4789, 3: 2379, 4: 841, 5: 204, 6: 41, 7: 11, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
                                     source_sentence  \
0  Suppose you were called to testify in court ag...   

                                     target_sentence  \
0  Reka tuvuge ko wahamagawe kujya gutanga ubuham...   

                                         translation    scores  tgt_len dist  
0  Reka tuvuge ko wahamagawe kujya gutanga ubuham... -3.374266       15    2  
(19999, 2) (80001, 2)


In [ ]:
split_srctgt('/data/rw/k4/rttl/rttl_train.csv','en', 'rw', 'rttl', 'k4', '2')

(49999, 2)


In [ ]:
generating_BPE('en', 'rw', 'rttl', 'k4', '2')

### Training $M$

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k4', '2')
! cd joeynmt; python3 -m joeynmt train configs/k4/transformer_reverse_enrw2_rttl.yaml

2022-10-15 01:09:32,316 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 01:09:32,573 - INFO - joeynmt.data - Loading training data...
2022-10-15 01:09:33,306 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 01:09:33,573 - INFO - joeynmt.data - Loading dev data...
2022-10-15 01:09:33,588 - INFO - joeynmt.data - Loading test data...
2022-10-15 01:09:33,955 - INFO - joeynmt.data - Data loaded.
2022-10-15 01:09:33,956 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 01:09:34,168 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 01:09:37,303 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 01:09:37,583 - INFO - joeynmt.training - Total params: 12154368
2022-10-15 01:09:41,270 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_rttl
2022-10-15 01:09:41,271 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 01:09:41,271 - INFO - joeynmt.helpers -    

# Kinyarwanda K5

## Iteration 1

### Training $M_{rev}$

In [ ]:
# English to Kinyarwanda
run_model('rw','en', 'rw','transformer_enrw1','random', 'k5', '1')
! cd joeynmt; python3 -m joeynmt train configs/k5/transformer_enrw1.yaml

2022-09-29 09:29:00,550 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-29 09:29:00,780 - INFO - joeynmt.data - Loading training data...
2022-09-29 09:29:03,034 - INFO - joeynmt.data - Building vocabulary...
2022-09-29 09:29:03,498 - INFO - joeynmt.data - Loading dev data...
2022-09-29 09:29:03,938 - INFO - joeynmt.data - Loading test data...
2022-09-29 09:29:04,717 - INFO - joeynmt.data - Data loaded.
2022-09-29 09:29:04,717 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-29 09:29:04,943 - INFO - joeynmt.model - Enc-dec model built.
2022-09-29 09:29:08,961 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-29 09:29:09,401 - INFO - joeynmt.training - Total params: 12140032
2022-09-29 09:29:12,942 - INFO - joeynmt.helpers -                           cfg.name : transformer_enrw1
2022-09-29 09:29:12,942 - INFO - joeynmt.helpers -                       cfg.data.src : en
2022-09-29 09:29:12,943 - INFO - joeynmt.helpers -                 

## Iteration 5

### Preprocessing

In [ ]:
!cp -a /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/nrttl/1/. /content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/rttl/1/

In [ ]:
rttl_data_load('rwtrain.csv', 'rw', 'rttl', 'k5', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/rw/k5
/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/rttl
(30000, 2)
{1: 11731, 2: 4772, 3: 2411, 4: 830, 5: 196, 6: 44, 7: 11, 8: 3, 9: 2, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
                      source_sentence  \
0  30 Is It Wrong to Use God’s Name ?   

                             target_sentence  \
0  30 Ese ni bibi gukoresha izina ry’Imana ?   

                                 translation    scores  tgt_len dist  
0  30 Ese ni bibi gukoresha izina ry’Imana ? -3.185903        8    1  
(20000, 2) (80000, 2)


In [ ]:
split_srctgt('/data/rw/k5/rttl/rttl_train.csv','en', 'rw', 'rttl', 'k5', '2')

(50000, 2)


In [ ]:
generating_BPE('en', 'rw', 'rttl', 'k5', '2')

### Training $M$

In [ ]:
# Swahili to English
run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k5', '2')
! cd joeynmt; python3 -m joeynmt train configs/k5/transformer_reverse_enrw2_rttl.yaml

2022-10-15 01:10:52,778 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 01:10:53,594 - INFO - joeynmt.data - Loading training data...
2022-10-15 01:10:54,278 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 01:10:54,522 - INFO - joeynmt.data - Loading dev data...
2022-10-15 01:10:54,538 - INFO - joeynmt.data - Loading test data...
2022-10-15 01:10:54,886 - INFO - joeynmt.data - Data loaded.
2022-10-15 01:10:54,886 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 01:10:55,092 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 01:10:59,155 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 01:10:59,552 - INFO - joeynmt.training - Total params: 12155136
2022-10-15 01:11:03,075 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enrw2_rttl
2022-10-15 01:11:03,075 - INFO - joeynmt.helpers -                       cfg.data.src : rw
2022-10-15 01:11:03,075 - INFO - joeynmt.helpers -    

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k5/transformer_reverse_enrw2_rttl/validations.txt"

Steps: 600	Loss: 130222.18750	PPL: 102.37486	bleu: 0.07842	LR: 0.00100000	*
Steps: 1200	Loss: 118399.35156	PPL: 67.24939	bleu: 0.38817	LR: 0.00100000	*
Steps: 1800	Loss: 109568.02344	PPL: 49.13176	bleu: 1.61682	LR: 0.00100000	*
Steps: 2400	Loss: 103041.87500	PPL: 38.96014	bleu: 2.65873	LR: 0.00100000	*
Steps: 3000	Loss: 98214.92188	PPL: 32.81773	bleu: 2.90704	LR: 0.00100000	*
Steps: 3600	Loss: 94853.19531	PPL: 29.12157	bleu: 3.22936	LR: 0.00100000	*
Steps: 4200	Loss: 91444.36719	PPL: 25.79847	bleu: 3.49581	LR: 0.00100000	*
Steps: 4800	Loss: 89008.93750	PPL: 23.65914	bleu: 4.31101	LR: 0.00100000	*
Steps: 5400	Loss: 87169.78906	PPL: 22.16199	bleu: 4.39495	LR: 0.00100000	*
Steps: 6000	Loss: 85566.14062	PPL: 20.93408	bleu: 4.55252	LR: 0.00100000	*
Steps: 6600	Loss: 83722.00781	PPL: 19.60589	bleu: 5.51775	LR: 0.00100000	*
Steps: 7200	Loss: 82459.25000	PPL: 18.74537	bleu: 5.62761	LR: 0.00100000	*
Steps: 7800	Loss: 81206.70312	PPL: 17.92911	bleu: 6.14318	LR: 0.00100000	*
Steps: 8400	Loss: 803

In [ ]:
# Reloading configuration file
ckpt_number = 25200
config = run_model('rw', 'rw','en','transformer_reverse_enrw2_rttl','rttl', 'k5', '2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k5/transformer_reverse_enrw2_rttl/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k5/transformer_reverse_enrw2_rttl"', f'model_dir: "models/k5/transformer_reverse_enrw2_rttl_reload"').replace(
        f'epochs: 60', f'epochs: 34')
        
with open("joeynmt/configs/k5/transformer_reverse_enrw2_rttl_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k5/transformer_reverse_enrw2_rttl_reload.yaml

2022-10-15 03:10:31,618 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 03:10:32,147 - INFO - joeynmt.data - Loading training data...
2022-10-15 03:10:36,725 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 03:10:37,572 - INFO - joeynmt.data - Loading dev data...
2022-10-15 03:10:38,962 - INFO - joeynmt.data - Loading test data...
2022-10-15 03:10:40,804 - INFO - joeynmt.data - Data loaded.
2022-10-15 03:10:40,804 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 03:10:41,015 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 03:10:45,525 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 03:10:45,790 - INFO - joeynmt.training - Total params: 12155136
2022-10-15 03:10:49,305 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k5/transformer_reverse_enrw2_rttl/25200.ckpt
2022-10-15 03:10:55,208 - INFO - joeynmt.helpers -                           cfg.name : tr

In [8]:
!cd joeynmt; python -m joeynmt test 'models/k5/transformer_reverse_enrw2_rttl_reload/config.yaml'

2022-10-17 10:11:17,738 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-17 10:11:19,492 - INFO - joeynmt.data - Building vocabulary...
2022-10-17 10:11:22,102 - INFO - joeynmt.data - Loading dev data...
2022-10-17 10:11:23,904 - INFO - joeynmt.data - Loading test data...
2022-10-17 10:11:30,214 - INFO - joeynmt.data - Data loaded.
2022-10-17 10:11:30,247 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2022-10-17 10:11:30,248 - INFO - joeynmt.prediction - Loading model from models/k5/transformer_reverse_enrw2_rttl_reload/57000.ckpt
2022-10-17 10:11:42,162 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-17 10:11:42,369 - INFO - joeynmt.model - Enc-dec model built.
2022-10-17 10:11:42,437 - INFO - joeynmt.prediction - Decoding on dev set (/content/gdrive/Shareddrives/Low_Budget_MT/data/rw/k5/rttl/2/dev.bpe.en)...
2022-10-17 10:12:21,655 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period (